# Scikit-Learn LogisticRegression
Using IRIS_VIEW from SAP Datasphere. This view has 150 records

## Install fedml_gcp package

In [ ]:
pip install fedml_gcp

## Import Libraries

In [ ]:
import os
import time
from fedml_gcp import dwcgcp

## Some constant variables to use throughout the notebook

In [ ]:
PROJECT_ID = '<project_id>'
REGION = '<region>'

BUCKET_NAME = '<bucket_name>'
BUCKET_URI = "gs://"+BUCKET_NAME
BUCKET_FOLDER = 'logisticReg'
MODEL_OUTPUT_DIR = BUCKET_URI+'/'+BUCKET_FOLDER
GCS_PATH_TO_MODEL_ARTIFACTS= MODEL_OUTPUT_DIR+'/model/'

TRAINING_PACKAGE_PATH = 'LogisticRegression'
JOB_NAME = "logistic-regression-training"

MODEL_DISPLAY_NAME = "logistic-regression-model"
DEPLOYED_MODEL_DISPLAY_NAME = 'logistic-regression-deployed-model'

TAR_BUNDLE_NAME = 'LogisticRegression.tar.gz'

CONTAINER_REGISTRY_REPOSITORY = 'logisticregression'
IMAGE = 'image-'+str(int(time.time()))

# Create DwcGCP Instance to access class methods and train model

It is expected that the bucket name passed here already exists in Cloud Storage.

For information on this constructor, please refer to the readme.

In [ ]:
params = {'project':PROJECT_ID,
         'location':REGION, 
         'staging_bucket':BUCKET_URI}

In [ ]:
dwc = dwcgcp.DwcGCP(params)


# Create tar bundle of script folder so GCP can use it for training

Please refer to the readme for more information on the dwc.make_tar_bundle() function

Before running this cell, please ensure that the script package has all the necessary files for a training job.

In [ ]:
dwc.make_tar_bundle(TAR_BUNDLE_NAME, 
                    TRAINING_PACKAGE_PATH, 
                    BUCKET_FOLDER+'/train/'+TAR_BUNDLE_NAME)


## Determine which training image and deploying image you want to use. 

Please refer here for the training pre-built containers: https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container

Please refer here for the deployment pre-built containers: https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers

In [ ]:
TRAIN_VERSION = <ADD YOUR TRAINING IMAGE HERE FROM THE GCP LINKS ABOVE>
DEPLOY_VERSION = <ADD YOUR DEPLOY IMAGE HERE FROM THE GCP LINKS ABOVE>

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

# Training using a custom python package and prebuilt container
For information on the dwc.train_model() function, please refer to the readme.

In the training inputs, we are using a script. When using a script, we have to pass the required packages needed as well.

We are also passing args which hold the table name to get data from and some other arguments we want to access in our training script. Before running the following cell, you should have a config.json uploaded to the bucket name you specified above with the path being /gcs/'+bucket_name+'/config.json'. This is specified in the training script, inside the function called get_dwc_data. This is used as the url parameter to DbConnection() so DbConnection knows where to find your credentials for access to SAP Datasphere.

You should also have the follow view IRIS_VIEW created in your SAP Datasphere. To gather this data, please refer to https://www.kaggle.com/uciml/iris

In [ ]:
table_name = 'IRIS_VIEW'
job_dir = 'gs://'+BUCKET_NAME

cmd_args = [
    "--table_name=" + str(table_name),
    "--job-dir=" + str(job_dir),
    "--table_size=" + '1',
    "--bucket_name=" + str(BUCKET_NAME),
    "--bucket_folder=" + str(BUCKET_FOLDER),
    "--package_name=" + 'trainer'
    
]

In [ ]:
inputs ={
    'display_name':JOB_NAME,
    'python_package_gcs_uri':BUCKET_URI + '/' + BUCKET_FOLDER+'/train/'+TAR_BUNDLE_NAME,
    'python_module_name':'trainer.task',
    'container_uri':TRAIN_IMAGE,
    'model_serving_container_image_uri':DEPLOY_IMAGE,
}

In [ ]:
run_job_params = {'model_display_name':MODEL_DISPLAY_NAME,
                  'args':cmd_args,
                  'replica_count':1,
                  'base_output_dir':MODEL_OUTPUT_DIR,
                  'sync':True}

In [ ]:
job = dwc.train_model(training_inputs=inputs, 
                      training_type='customPythonPackage',
                     params=run_job_params)

## Deployment

For information on the dwc.deploy() function please refer to the readme.

Here we are deploying a custom predictor for the model we trained above.

In [ ]:
model_config = {
    'deployed_model_display_name': DEPLOYED_MODEL_DISPLAY_NAME,
    'traffic_split':{"0": 100},
    'machine_type':'n1-standard-2',
    'min_replica_count':1,
    'max_replica_count':1,
    'sync':True
    
}
deployed_endpoint = dwc.deploy(model=job, model_config=model_config)

## Prediction

Once the model is deployed to an endpoint, we can run predictions on it.

For information on the dwc.predict() function please refer to the readme.

Since we are using DbConnection here, we will need to have the config.json in this notebook instance as well.

In [ ]:
import os
import pandas as pd
import numpy as np
from fedml_gcp import DbConnection

In [ ]:
db = DbConnection()
org_data = db.get_data_with_headers(table_name='IRIS_VIEW', size=float(1))
org_data = pd.DataFrame(org_data[0], columns=org_data[1])
org_data = org_data.sample(frac=1).reset_index(drop=True)
org_data = org_data[101:]
org_data

In [ ]:
x_data = org_data.drop(['species'], axis=1)
x_data

In [ ]:
params = {
    'instances': x_data.astype('float64').values.tolist()
}

response = dwc.predict(predict_params=params, endpoint=deployed_endpoint)

In [ ]:
LABEL_TO_INDEX = {
    'Iris-virginica': 0,
    'Iris-versicolor': 1,
    'Iris-setosa': 2
}
org_data['species'].map(LABEL_TO_INDEX).values

In [ ]:
len(response)

In [ ]:
actuals = org_data['species'].map(LABEL_TO_INDEX).to_numpy()

In [ ]:
from sklearn import metrics
acc = metrics.accuracy_score(actuals, response)
acc